In [32]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import string
import re
import nltk
import enchant
import sklearn

%matplotlib inline

In [23]:
data = pd.read_csv('finaldata.csv')

In [24]:
data.head()

,Category,Url,Text
0,Agriculture,http://www.cartercenter.org/health/agriculture...,the carter center waging peace fighting dise...
1,Agriculture,http://www.agriculturalproductsindia.com/,home introduction history technology agro as...
2,Agriculture,http://www.sare.org/,advanced search mysare login social media...
3,Agriculture,http://www.cias.wisc.edu/,about people contact donate join our email ...
4,Agriculture,http://www.globalservicecorps.org/site/tanzani...,home sitemap what sets gsc apart continuum of...


In [9]:
###############Randomizing sequence of data
data = data.iloc[np.random.permutation(len(data))]
data = data.reset_index(drop=True)

In [15]:
d = enchant.Dict("en_US")
def removenonsensewords(text):
    tokens = nltk.word_tokenize(text)
    
    stemmed = []
    #i=0
    for token in tokens:
        #print(i)
        #i=i+1
        if d.check(token):
            stemmed.append(token)
        
    return ' '.join(stemmed)

data['Text_Dictionary']= data['Text'].apply(removenonsensewords)

In [14]:
def listofbadwords():
    from nltk.corpus import stopwords
    stopwords = stopwords.words('english')
    monthnames = ['jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec']
    randomrepetitive = ['edu','unl','mt']
    
    totlist = stopwords + monthnames + randomrepetitive
    return totlist

totlist = listofbadwords()
def removebadwords(x):
    
    wordlist = x.split()
    wordlist = [word for word in wordlist if word.lower() not in totlist]
    x = ' '.join(wordlist)
    return x
data['Text'] = data['Text'].apply(removebadwords)

In [19]:
data[['Text','Text_Dictionary']].head()

,Text,Text_Dictionary
0,introduction agri tech personnel recruitment s...,introduction tech personnel recruitment servic...
1,login place classified submit news post events...,login place classified submit news post events...
2,irish american information service irish ameri...,information service information service non pr...
3,offices contact us site map log industry centr...,offices contact us site map log industry facts...
4,home believe quality changes world display wor...,home believe quality changes world display wor...


In [17]:
data.to_csv('removedstopwords.csv',index=False)

In [25]:
data = pd.read_csv('removedstopwords.csv')

In [27]:
data = data.dropna()

In [50]:
from sklearn.feature_extraction.text import HashingVectorizer

vect = HashingVectorizer(norm = None,n_features=1000)
train_tfidf = vect.fit_transform(data['Text_Dictionary'])

In [51]:
train_tfidf.shape

(32241, 1000)

In [53]:
train_tfidf.toarray()

array([[ 0.,  0.,  0., ...,  0.,  0., -1.],
       [ 0.,  0.,  0., ...,  3.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 2.,  0.,  0., ...,  0.,  0., -7.],
       [ 0.,  0.,  0., ...,  1.,  0.,  0.]])

In [58]:
from sklearn.cross_validation import train_test_split
train_tfidf = train_tfidf.toarray()
data_train,data_validate,labels_train,labels_validate = train_test_split(train_tfidf, data['Category'],test_size = 0.2)

In [63]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(data_train,labels_train)
sklearn.metrics.accuracy_score(model.predict(data_validate),labels_validate)

0.95735772988060164

In [49]:
ch2 = sklearn.feature_selection.SelectKBest(sklearn.feature_selection.chi2, k = 1000)
train_vec5000 = ch2.fit_transform(train_tfidf, data['Category'])
train_vec5000.shape

ValueError: Input X must be non-negative.

In [95]:
from bs4 import BeautifulSoup
import requests
import string
def removepunctuation(x):
    #x = x.replace('.',' ')
    #x = x.replace(')',' ')
    #x = x.replace('(',' ')
    replace_punctuation = string.maketrans(string.punctuation, ' '*len(string.punctuation))
    x = x.translate(replace_punctuation)
    #retstr = x.translate(string.maketrans("",""), string.punctuation)
    return x
    
def removeunicode(x):
    return re.sub(r'[^\x00-\x7F]+',' ', x)
def lowercasestring(x):
    return x.lower()

def removedigits(s):
    s = re.sub(" \d+", " ", s)
    return s
    
def cleanstring(x):
    #x=replaceredundancy(x)
    #x=removepunctuation(x)
    x=removeunicode(x)
    #x = trimstring(x)
    x=removedigits(x)
    x=lowercasestring(x)
    return x 

def getprediction(url):
    website = requests.get(url)
    soup = BeautifulSoup(website.content)
    [s.extract() for s in soup(['style', 'script', '[document]', 'head', 'title'])]
    text = soup.getText()
    temp = re.sub(r'[\n\t\r ]+', " ", text)
    temp= re.sub(r'\s+'," ",temp)
    temp =  re.sub(r'\s+'," ",temp)
    
    temp = cleanstring(temp)
    
    lis = []
    lis.append(temp)
    #print(temp)
    feature_text = vect.transform(lis)
    
    
    pred = model.predict(feature_text)
    print(feature_text.shape)
    
    return pred


In [105]:
getprediction("http://www.aid-diagnostika.com/")

(1, 1000)


array(['Agriculture'], dtype=object)

In [104]:
getprediction("http://economictimes.indiatimes.com/news/economy/agriculture/indians-go-for-cheaper-pulses-as-tur-dal-prices-keep-rising/articleshow/53042439.cms")

(1, 1000)


array(['News'], dtype=object)